In [56]:
from __future__ import division
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import requests
import ast

In [57]:
V_T_df=pd.read_csv('data/spark_V.csv')
id_map=pd.read_csv('data/id_mapping.csv')
avg_show=pd.read_csv('data/avg_show_R.csv')
# df = pd.read_csv('data/V_T.csv')

In [58]:
item_mapping=np.array(id_map['col1'])

In [59]:
avg_show.head()

,Unnamed: 0,anime_id,avg_for_show
0,0,1,85.094644
1,1,5,80.694702
2,2,6,79.888049
3,3,7,71.701940
4,4,8,70.015306


In [60]:
url='https://anilist.co/api/'
cid='selzla-6acux'
sec='eGi4fmsY9pV64E1fSTWJJ1'
username='selzla'
params={'grant_type':"client_credentials",'client_id':cid,'client_secret':sec}
access=requests.post(url+'auth/access_token',data=params)
access_token=access.json()['access_token']

In [61]:
user_anime=requests.get(url+'user/'+username+'/animelist?access_token='+access_token)

In [62]:
completed = user_anime.json()['lists']['completed']

if 'watching' in user_anime.json()['lists']:
    watching = user_anime.json()['lists']['watching']
else:
    watching = []

if 'dropped' in user_anime.json()['lists']:
    dropped = user_anime.json()['lists']['dropped']
else:
    dropped = []

if 'on_hold' in user_anime.json()['lists']:
    on_hold = user_anime.json()['lists']['on_hold']
else:
    on_hold = []

if 'plan_to_watch' in user_anime.json()['lists']:
    plan_to_watch = user_anime.json()['lists']['plan_to_watch']
else:
    plan_to_watch = []

In [63]:
scores = []
for i in range(len(completed)):
    title=completed[i]['anime']['title_english'].encode("utf-8")
    anime_id=completed[i]['anime']['id']
    score=completed[i]['score_raw']
    scores.append([anime_id, score])

In [64]:
series_df=pd.read_csv('data/series_data.csv')

In [65]:
series_df['real_genres']=series_df['genres'].apply(lambda x: ast.literal_eval(x))

In [66]:
def isAdult(ID):
    if len(list(series_df[series_df['id']==ID]['adult'])):
        return list(series_df[series_df['id']==ID]['adult'])[0]
    else:
        return None

In [67]:
def hasGenre(ID, genre):
    if len(list(series_df[series_df['id']==ID]['real_genres']))>0:
        return genre in list(series_df[series_df['id']==ID]['real_genres'])[0]
    else:
        return None

In [68]:
def StartYear(ID):
    if len(list(series_df[series_df['id']==ID]['start_date']))>0:
        return int(list(series_df[series_df['id']==ID]['start_date'])[0][:4])
    else:
        return None

In [69]:
def numEpisodes(ID):
    if len(list(series_df[series_df['id']==ID]['total_episodes']))>0:
        return list(series_df[series_df['id']==ID]['total_episodes'])[0]
    else: 
        return None

In [70]:
def Type(ID):
    if len(list(series_df[series_df['id']==ID]['type']))>0:
        return list(series_df[series_df['id']==ID]['type'])[0]
    else:
        return None

In [71]:
len(series_df)

10286

In [72]:
titles={}
for i in range(len(series_df)):
    ID = series_df['id'][i]
    title = series_df['title_english'][i]
    titles[ID] = title


In [73]:
user=np.array([row for row in scores if row[0] in item_mapping])
#user=np.array([[row[0]-1, row[1]] for row in user])
# print user

In [74]:
item_mapping[1]

5

In [75]:
V=np.array(V_T_df).T
print V.shape[0], V.shape[1]

10 9856


In [76]:
average_ratings={}
for i in range(len(avg_show)):
    average_ratings[avg_show['anime_id'][i]]=avg_show['avg_for_show'][i]
print average_ratings[5]

80.6947019868


In [77]:
user_vector = np.zeros(V.shape[1])
for row in user:
    user_vector[list(item_mapping).index(row[0])]=row[1]+74-np.mean(user[:,1])-average_ratings[row[0]]

In [78]:
new_user=user_vector[np.where(user_vector!=0)]
new_V=V.T[np.where(user_vector!=0)].T
s=np.linalg.lstsq(new_V.T,new_user.T)[0].T
new_R=np.dot(s,V)
for i in range(len(new_R)):
    new_R[i]=new_R[i]-74+np.mean(user[:,1])+average_ratings[item_mapping[i]]

In [79]:
# print s
# print V.T[list(new_R).index(max(new_R))]
max(new_R)

146.26639993732462

In [80]:
seen = [row[0] for row in scores]
others = [watching, dropped, on_hold, plan_to_watch]
for x in others:
    for i in range(len(x)):
        seen.append(x[i]['anime']['id'])

In [81]:
rec=np.argsort(-1*new_R)
rec=[item_mapping[x] for x in rec] 
f_type = (True, 'TV')
f_genres = (False, 'Slice of Life')
f_year = (False, 2000, 2016)
f_episodes = (False, 1, 100)
f_adult = (False, False)
count = 0
for i in range(10000):
    if rec[i] not in seen:
        allowed = True
        if f_type[0]:
            if Type(rec[i])!=f_type[1]:
                allowed = False
        if f_genres[0]:
            if not hasGenre(rec[i], f_genres[1]):
                allowed = False
        if f_year[0]:
            if StartYear(rec[i])<f_year[1] or StartYear(rec[i])>f_year[2]:
                allowed = False
        if f_episodes[0]:
            if numEpisodes(rec[i])<f_episodes[1] or numEpisodes(rec[i])>f_episodes[2]:
                allowed = False 
        if f_adult[0]:
            if isAdult(rec[i])!=f_adult[1]:
                allowed = False
        if allowed:
            print rec[i], titles[rec[i]]
            count +=1 
    if count>15:
        break
    
    

4411 Future Robo Daltanius
4616 Yumemiru Toppo Jijo
12863 Pretty Rhythm: Dear My Future
3837 Mashin Eiyuuden Wataru
14333 Tanken Driland
2337 Tetsujin 28
6149 Little Miss Maruko
20655 Hero Bank
20906 LUPIN THE 3rd PART4
3075 The Flying House
3036 Soar High! Isami
5474 Yobarete Tobidete Akubi-chan
2232 Funny Pets
17497 Jewelpet Happiness
6055 F
4807 Spirit Hero Wataru 2


In [82]:
G = []
for x in series_df['real_genres']:
    for y in x:
        if y not in G:
            G.append(y)
print G

['Hentai', 'Sports', 'Comedy', 'Ecchi', 'Drama', 'Mystery', 'Romance', 'Supernatural', 'Fantasy', 'Mahou Shoujo', 'Slice of Life', 'Action', 'Mecha', 'Adventure', 'Sci-Fi', 'Music', '', 'Horror', 'Psychological', 'Thriller']


In [83]:
np.argsort(-1*new_R)

array([3619, 5962, 3733, ..., 9796, 2138, 3571])

In [84]:
V.T[9295]

array([ 0.5803225 ,  3.54606748,  1.93732285,  0.32532278, -4.63045263,
        0.55133301, -1.32415223,  0.13227721, -1.53926277,  2.87482834])